In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, "..")
from paths import *
from neo4j import GraphDatabase, basic_auth


In [141]:
compound_type = "targeted"
sample = "feces"
sel_sheet_index = 0

filename = "extracted_paths_for_{}_compounds_{}_sample_sheet_index_{}_dict.pickle".format(compound_type, sample, str(sel_sheet_index))


In [302]:
with open(os.path.join(OUTPUT_PATH, filename), "rb") as f:
    extracted_paths = pickle.load(f)
    

In [303]:
compound_to_intermediate_path = extracted_paths["compound_to_intermediate"]
starting_nodes = list(compound_to_intermediate_path.keys())
tot_intermediate_nodes = len(compound_to_intermediate_path[starting_nodes[0]])

max_hop = 1
intermediate_nodes_sel = []
intermediate_nodes_sel_dict_list = []
for starting_node in starting_nodes:
    intermediate_node_hops = np.array(list(map(lambda x:x.shape[0], compound_to_intermediate_path[starting_node])))
    intermediate_nodes_sel.append(np.where(intermediate_node_hops <= max_hop)[0])
    intermediate_nodes_sel_dict_list.append(
                                    {
                                        starting_node : np.where(intermediate_node_hops <= max_hop)[0]
                                    }
                                  )

intermediate_nodes_sel = np.concatenate(intermediate_nodes_sel)
intermediate_nodes_sel = np.unique(intermediate_nodes_sel)

print("Total MS-proximal salient intermediate nodes = ", tot_intermediate_nodes)
print("Total MS-proximal salient intermediate nodes with <= {} hop connections with starting compound nodes = {}".format(max_hop, len(intermediate_nodes_sel)))


Total MS-proximal salient intermediate nodes =  206
Total MS-proximal salient intermediate nodes with <= 1 hop connections with starting compound nodes = 88


In [460]:
total_intermediate_nodes_captured = []
total_intermediate_paths_captured = []
for index, starting_node in enumerate(starting_nodes):
    intermediate_nodes_sel = intermediate_nodes_sel_dict_list[index][starting_node]
    for intermediate_nodes_sel_ in intermediate_nodes_sel:
        total_intermediate_nodes_captured.append(compound_to_intermediate_path[starting_node][intermediate_nodes_sel_].target.values)
        total_intermediate_paths_captured.append(compound_to_intermediate_path[starting_node][intermediate_nodes_sel_])
total_intermediate_nodes_captured = np.unique(np.concatenate(total_intermediate_nodes_captured))
total_intermediate_paths_captured = pd.concat(total_intermediate_paths_captured, ignore_index=True).drop_duplicates()


In [465]:
ms_proximal_salient_intermediate_nodes = list(intermediate_to_MS_path.keys())
ms_proximal_salient_intermediate_nodes_captured = []
for item in total_intermediate_nodes_captured:
    if item in ms_proximal_salient_intermediate_nodes:
        ms_proximal_salient_intermediate_nodes_captured.append(item)
        

In [466]:
intermediate_to_MS_path = extracted_paths["intermediate_to_MS"]
intermediate_to_MS_path_captured = []
for item in ms_proximal_salient_intermediate_nodes_captured:
    intermediate_to_MS_path_captured.append(intermediate_to_MS_path[item])
intermediate_to_MS_path_captured = pd.concat(intermediate_to_MS_path_captured, ignore_index=True).drop_duplicates()
    


In [467]:
MS_id = "Disease:DOID:2377"


total_intermediate_paths_captured.loc[:, "source_stage"] = 3
total_intermediate_paths_captured.loc[:, "target_stage"] = 3
total_intermediate_paths_captured.loc[total_intermediate_paths_captured.source.isin(starting_nodes), "source_stage"] = 1
total_intermediate_paths_captured.loc[total_intermediate_paths_captured.target.isin(starting_nodes), "target_stage"] = 1
total_intermediate_paths_captured.loc[total_intermediate_paths_captured.source == MS_id, "source_stage"] = 4
total_intermediate_paths_captured.loc[total_intermediate_paths_captured.target == MS_id, "target_stage"] = 4
total_intermediate_paths_captured.loc[total_intermediate_paths_captured.source.isin(ms_proximal_salient_intermediate_nodes), "source_stage"] = 2
total_intermediate_paths_captured.loc[total_intermediate_paths_captured.target.isin(ms_proximal_salient_intermediate_nodes), "target_stage"] = 2

intermediate_to_MS_path_captured.loc[:, "source_stage"] = 3
intermediate_to_MS_path_captured.loc[:, "target_stage"] = 3
intermediate_to_MS_path_captured.loc[intermediate_to_MS_path_captured.source.isin(starting_nodes), "source_stage"] = 1
intermediate_to_MS_path_captured.loc[intermediate_to_MS_path_captured.target.isin(starting_nodes), "target_stage"] = 1
intermediate_to_MS_path_captured.loc[intermediate_to_MS_path_captured.source == MS_id, "source_stage"] = 4
intermediate_to_MS_path_captured.loc[intermediate_to_MS_path_captured.target == MS_id, "target_stage"] = 4
intermediate_to_MS_path_captured.loc[intermediate_to_MS_path_captured.source.isin(ms_proximal_salient_intermediate_nodes), "source_stage"] = 2
intermediate_to_MS_path_captured.loc[intermediate_to_MS_path_captured.target.isin(ms_proximal_salient_intermediate_nodes), "target_stage"] = 2

total_intermediate_paths_captured


,source,target,source_stage,target_stage
0,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Compound:CHEBI:77746,1,2
1,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Compound:CHEBI:76969,1,2
2,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Protein:A6T9V6,1,2
3,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Protein:A4YZD7,1,2
4,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Protein:B4EFI5,1,2
...,...,...,...,...
195,Compound:inchikey:XBDQKXXYIPTUBI-UHFFFAOYSA-N,Protein:P37288,1,2
196,Compound:inchikey:XBDQKXXYIPTUBI-UHFFFAOYSA-N,Protein:P34995,1,2
197,Compound:inchikey:XBDQKXXYIPTUBI-UHFFFAOYSA-N,Protein:P47900,1,2
198,Compound:inchikey:XBDQKXXYIPTUBI-UHFFFAOYSA-N,Protein:P25116,1,2


In [469]:
final_extracted_path = pd.concat([total_intermediate_paths_captured, intermediate_to_MS_path_captured], ignore_index=True).drop_duplicates()

final_extracted_path


,source,target,source_stage,target_stage
0,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Compound:CHEBI:77746,1,2
1,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Compound:CHEBI:76969,1,2
2,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Protein:A6T9V6,1,2
3,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Protein:A4YZD7,1,2
4,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Protein:B4EFI5,1,2
...,...,...,...,...
299,ProteinDomain:PF02518,Compound:inchikey:KFZMGEQAYNKOFK-UHFFFAOYSA-N,2,3
300,ProteinDomain:PF07690,Compound:inchikey:ZQPPMHVWECSIRJ-KTKRTIGZSA-N,2,3
301,Compound:inchikey:ZQPPMHVWECSIRJ-KTKRTIGZSA-N,Disease:DOID:2377,3,4
302,ProteinDomain:PF12697,Compound:inchikey:HPNSFSBZBAHARI-RUDMXATFSA-N,2,3


In [321]:
%%time

nodes_df = pd.DataFrame(pd.concat([final_extracted_path.source, final_extracted_path.target]), columns=["node_id"]).drop_duplicates()
nodes_df["node_type"] = nodes_df.node_id.apply(lambda x:x.split(":")[0])
nodes_df["node_id"] = nodes_df.node_id.apply(lambda x:":".join(x.split(":")[1:]))

query = """
            MATCH(n:{})
            WHERE n.identifier = '{}'
            RETURN n.name AS n_name
        """

auth = basic_auth(SPOKE_USER, SPOKE_PASSWORD)
sdb = GraphDatabase.driver(URI, auth=auth)

node_list = []
with sdb.session() as session:
    with session.begin_transaction() as tx:
        for index, row in nodes_df.iterrows():
            result = tx.run(query.format(row["node_type"], row["node_id"]))
            for row_ in result:
                node_list.append((row["node_id"], row_["n_name"], row["node_type"]))
sdb.close()

nodes_df = pd.DataFrame(node_list, columns = ["node_id", "node_name", "node_type"])
nodes_df["composite_id"] = nodes_df["node_type"] + ":" + nodes_df["node_id"]
nodes_df


CPU times: user 277 ms, sys: 26.3 ms, total: 304 ms
Wall time: 3.65 s


,node_id,node_name,node_type,composite_id
0,inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,2-Methylbutanoic acid,Compound,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N
1,inchikey:QTBSBXVTEAMEQO-UHFFFAOYSA-N,Acetic Acid,Compound,Compound:inchikey:QTBSBXVTEAMEQO-UHFFFAOYSA-N
2,inchikey:FERIUCNNQQJTOY-UHFFFAOYSA-N,Butyric Acid,Compound,Compound:inchikey:FERIUCNNQQJTOY-UHFFFAOYSA-N
3,inchikey:GWYFCOCPABKNJV-UHFFFAOYSA-N,Isovaleric acid,Compound,Compound:inchikey:GWYFCOCPABKNJV-UHFFFAOYSA-N
4,inchikey:XBDQKXXYIPTUBI-UHFFFAOYSA-N,Propionic Acid,Compound,Compound:inchikey:XBDQKXXYIPTUBI-UHFFFAOYSA-N
...,...,...,...,...
105,PF07690,Major Facilitator Superfamily,ProteinDomain,ProteinDomain:PF07690
106,inchikey:ZQPPMHVWECSIRJ-KTKRTIGZSA-N,Oleic Acid,Compound,Compound:inchikey:ZQPPMHVWECSIRJ-KTKRTIGZSA-N
107,PF12697,Alpha/beta hydrolase family,ProteinDomain,ProteinDomain:PF12697
108,inchikey:HPNSFSBZBAHARI-RUDMXATFSA-N,Mycophenolic acid,Compound,Compound:inchikey:HPNSFSBZBAHARI-RUDMXATFSA-N


In [323]:
final_extracted_path_1 = pd.merge(final_extracted_path, nodes_df, left_on="source", right_on="composite_id")

final_extracted_path_2 = pd.merge(final_extracted_path, nodes_df, left_on="target", right_on="composite_id")

final_extracted_path_3 = pd.merge(final_extracted_path_1, final_extracted_path_2, on=["source", "target"])[["source", "target", "node_name_x", "node_name_y", "node_type_x", "node_type_y", "source_stage_x", "target_stage_x"]]

final_extracted_path_3.rename(columns={"source":"source_id", "target":"target_id", "node_name_x":"source_name", "node_name_y":"target_name", "node_type_x": "source_node_type", "node_type_y":"target_node_type", "source_stage_x":"source_stage", "target_stage_x":"target_stage"}, inplace=True)

final_extracted_path_3.drop_duplicates(inplace=True)

final_extracted_path_3


,source_id,target_id,source_name,target_name,source_node_type,target_node_type,source_stage,target_stage
0,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Compound:CHEBI:77746,2-Methylbutanoic acid,human metabolite,Compound,Compound,1,2
1,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Compound:CHEBI:76969,2-Methylbutanoic acid,bacterial metabolite,Compound,Compound,1,2
2,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Protein:A6T9V6,2-Methylbutanoic acid,A6T9V6_KLEP7,Compound,Protein,1,2
3,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Protein:A4YZD7,2-Methylbutanoic acid,A4YZD7_BRASO,Compound,Protein,1,2
4,Compound:inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,Protein:B4EFI5,2-Methylbutanoic acid,B4EFI5_BURCJ,Compound,Protein,1,2
...,...,...,...,...,...,...,...,...
299,ProteinDomain:PF02518,Compound:inchikey:KFZMGEQAYNKOFK-UHFFFAOYSA-N,"Histidine kinase-, DNA gyrase B-, and HSP90-li...",Isopropyl Alcohol,ProteinDomain,Compound,2,3
300,ProteinDomain:PF07690,Compound:inchikey:ZQPPMHVWECSIRJ-KTKRTIGZSA-N,Major Facilitator Superfamily,Oleic Acid,ProteinDomain,Compound,2,3
301,Compound:inchikey:ZQPPMHVWECSIRJ-KTKRTIGZSA-N,Disease:DOID:2377,Oleic Acid,multiple sclerosis,Compound,Disease,3,4
302,ProteinDomain:PF12697,Compound:inchikey:HPNSFSBZBAHARI-RUDMXATFSA-N,Alpha/beta hydrolase family,Mycophenolic acid,ProteinDomain,Compound,2,3


In [377]:

node_filename = "network_nodes_for_{}_compounds_{}_sample_sheet_index_{}.tsv".format(compound_type, sample, str(sel_sheet_index))
edge_filename = "network_edges_for_{}_compounds_{}_sample_sheet_index_{}.tsv".format(compound_type, sample, str(sel_sheet_index))

nodes_df.to_csv(os.path.join(OUTPUT_PATH, node_filename), sep="\t", index=False, header=True)
final_extracted_path_3.to_csv(os.path.join(OUTPUT_PATH, edge_filename), sep="\t", index=False, header=True)


In [357]:
ms_proximal_salient_intermediate_nodetypes = list(set(list(map(lambda x:x.split(":")[0], ms_proximal_salient_intermediate_nodes_captured))))
total_intermediate_nodetypes = len(ms_proximal_salient_intermediate_nodetypes)
print("There are a total of {} intermediate nodetypes".format(total_intermediate_nodetypes))
ms_proximal_salient_intermediate_nodetypes


There are a total of 4 intermediate nodetypes


['Compound', 'Food', 'ProteinDomain', 'Protein']

In [391]:
sel_intermediate_index = 3

ms_proximal_salient_intermediate_nodetype_sel = ms_proximal_salient_intermediate_nodetypes[sel_intermediate_index]
edge_filename_intermediate = "network_edges_for_{}_compounds_{}_sample_sheet_index_{}_intermediate_{}_{}.tsv".format(compound_type, sample, sel_sheet_index, sel_intermediate_index+1, total_intermediate_nodetypes)
final_extracted_path_3_1 = final_extracted_path_3[(
                            (final_extracted_path_3["target_stage"]==2) 
                               & 
                            (final_extracted_path_3["target_node_type"]==ms_proximal_salient_intermediate_nodetype_sel)
                        )
                      |(
                            (final_extracted_path_3["source_stage"]==2)
                               &
                            (final_extracted_path_3["source_node_type"]==ms_proximal_salient_intermediate_nodetype_sel)
                        )
                      ]

final_extracted_path_3_2 = final_extracted_path_3[(final_extracted_path_3.source_id.isin(final_extracted_path_3_1.target_id.values)) 
                       & 
                       (final_extracted_path_3.target_id==MS_id)]

final_extracted_path_3_ = pd.concat([final_extracted_path_3_1, final_extracted_path_3_2], ignore_index=True)




In [ ]:
# %%time
# GRAPH_PATH = "../spoke_35M.gpickle"
# with open(GRAPH_PATH, "rb") as f:
#     G = pickle.load(f)

In [392]:
edgetype_list = []
for index, row in final_extracted_path_3_.iterrows():
    edgetype_list.append(G[row["source_id"]][row["target_id"]]["edgetype"])
    
final_extracted_path_3_["edgetype"] = edgetype_list

final_extracted_path_3_.to_csv(os.path.join(OUTPUT_PATH, edge_filename_intermediate), sep="\t", index=False, header=True)
    

In [473]:
final_extracted_path_3_.edgetype.unique()

array(['INTERACTS_PiC', 'BINDS_CbP', 'TREATS_CtD'], dtype=object)